In [1]:
# import importlib

import numpy as np
import datetime

import plugins.custom_config as custom_config
import plugins.bot_tool as bot_tool
# from bot_tool import *
import plugins.dice_tool as dice_tool
import plugins.help_info as help_info
# from dice_tool import *
import plugins.utils as utils

# importlib.reload(bot_tool)
# importlib.reload(dice_tool)
# importlib.reload(utils)

SyntaxError: invalid syntax (__init__.py, line 119)

In [ ]:
assert utils.ChineseToEnglishSymbol('。') == '.'
assert utils.ChineseToEnglishSymbol('.') == '.'
assert utils.int2str(4) == '+4'
assert utils.int2str(0) == '+0'
assert utils.int2str(-1) == '-1'

assert bot_tool.ParseInput('.rd20').equal(utils.Command(utils.CommandType.Roll, ['d20', '', False]))
assert bot_tool.ParseInput(' . r D20 reason').equal(utils.Command(utils.CommandType.Roll, ['d20', 'reason', False]))
assert bot_tool.ParseInput('.r').equal(utils.Command(utils.CommandType.Roll, ['', '', False]))
assert bot_tool.ParseInput('abc') is None
assert bot_tool.ParseInput('') is None
assert bot_tool.ParseInput('测试') is None

In [ ]:
assert dice_tool.RollDiceCommond('12') [2][0] == 12 and dice_tool.RollDiceCommond('12') [1] == '12=12'
assert dice_tool.RollDiceCommond('4d6+2')[0] == 0 and dice_tool.RollDiceCommond('4d6+2') [1][:6] == '4D6+2='
assert dice_tool.RollDiceCommond('d20') [0] == 0 and dice_tool.RollDiceCommond('d20') [1][:3] == 'D20'
assert dice_tool.RollDiceCommond('d') [0] == 0 and dice_tool.RollDiceCommond('d') [1][:3] == 'D20'
assert dice_tool.RollDiceCommond('1D4') [0] == 0
assert dice_tool.RollDiceCommond('30D4') [0] == 0
assert dice_tool.RollDiceCommond('D20优势+2') [0] == 0 and dice_tool.RollDiceCommond('D20优势+2') [1][:12] == 'D20优势+2=Max('
assert dice_tool.RollDiceCommond('D20劣势-4') [0] == 0 and dice_tool.RollDiceCommond('D20劣势-4') [1][:12] == 'D20劣势-4=Min('
assert dice_tool.RollDiceCommond('4d6k3') [0] == 0 and dice_tool.RollDiceCommond('4d6k3') [1][:11] == '4D6K3=Max3('
assert dice_tool.RollDiceCommond('6#4d6k3') [0] == 0 and dice_tool.RollDiceCommond('6#4d6k3') [1][:14] == '6次4D6K3={\nMax3'
assert dice_tool.RollDiceCommond('2d20k1+5+1d4') [0] == 0 and dice_tool.RollDiceCommond('2d20k1+5+1d4') [1][:17] == '2D20K1+5+1D4=Max('
assert dice_tool.RollDiceCommond('d优势+2')[1][:12] == 'D20优势+2=Max('
assert dice_tool.RollDiceCommond('3dk1+5抗性')[1][:16] == '3D20K1+5抗性=(Max('

In [ ]:
assert dice_tool.SplitDiceCommand('d20优势+5攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20 优 势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20 优 势+5') == ('d20优势+5', '')

In [ ]:
b = bot_tool.Bot()
assert b.ProcessInput('.nn', '12345678', 'ABCD').resultStr == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER').resultStr[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd', '12345678', 'TEST_USER').resultStr[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd     ', '12345678', 'TEST_USER').resultStr[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd    未知原因', '12345678', 'TEST_USER').resultStr[:23] == '由于未知原因,TEST_USER掷出了D20='
assert b.ProcessInput('.rd20攻击', '12345678', 'TEST_USER', 'groupId_test').resultStr[:21] == '由于攻击,TEST_USER掷出了D20='
assert b.ProcessInput('.rd优势', '12345678', 'TEST_USER').resultStr[:22] == 'TEST_USER掷出了D20优势=Max('
assert b.ProcessInput('.rd+5', '12345678', 'TEST_USER').resultStr[:18] == 'TEST_USER掷出了D20+5='

In [ ]:
assert b.ProcessInput('.nn', '12345678', 'ABCD').resultStr == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.nn 西瓜', '12345678', 'ABCD').resultStr == '要称呼ABCD为西瓜吗? 没问题!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER').resultStr[:9] == '西瓜掷出了D20='
assert b.ProcessInput('.nn 冬瓜', '12345678', 'ABCD').resultStr == '要称呼ABCD为冬瓜吗? 没问题!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER').resultStr[:9] == '冬瓜掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER').resultStr[:10] == '冬瓜今天走运的概率是'
assert b.ProcessInput('.nn', '12345678', 'ABCD').resultStr == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER').resultStr[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER').resultStr[:17] == 'TEST_USER今天走运的概率是'

In [ ]:
assert b.ProcessInput('.nn 格鲁特', '12345678', 'TEST_USER').resultStr == '要称呼TEST_USER为格鲁特吗? 没问题!'
assert b.ProcessInput('.sethp 40', '12345678', 'TEST_USER', 'groupId_test').resultStr == '成功将格鲁特的生命值设置为40=40'
assert b.ProcessInput('.sethp 40/50', '12345678', 'TEST_USER', 'groupId_test').resultStr == '成功将格鲁特的生命值设置为40=40, 最大生命值是50'
assert b.ProcessInput('.init clr', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr == '还没有做好先攻列表哦'
assert b.ProcessInput('.ri+2', '12345678', 'TEST_USER', 'groupId_test').resultStr[:12] == '格鲁特先攻:D20+2='
assert b.ProcessInput('.ri-1', '12345678', 'TEST_USER', 'groupId_test').resultStr[:12] == '格鲁特先攻:D20-1='
assert b.ProcessInput('.ri', '12345678', 'TEST_USER', 'groupId_test').resultStr[:10] == '格鲁特先攻:D20='
assert b.ProcessInput('.ri-2地精', '12345678', 'TEST_USER', 'groupId_test').resultStr[:11] == '地精先攻:D20-2='
assert b.ProcessInput('.ri劣势+1d4 地精', '12345678', 'TEST_USER', 'groupId_test').resultStr[:19] =='地精先攻:D20劣势+1D4=Min('
assert b.ProcessInput('.ri20 巢穴', '12345678', 'TEST_USER', 'groupId_test').resultStr == '巢穴先攻:20=20'
assert b.ProcessInput('.ri-5', '87654321', '米达尔', 'groupId_test').resultStr[:12] == '米达尔先攻:D20-5='
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr)
assert b.ProcessInput('.init del 地精', '12345678', 'TEST_USER', 'groupId_test').resultStr == '已经将地精从先攻列表中删除'
assert b.ProcessInput('.init del米达尔', '12345678', 'TEST_USER', 'groupId_test').resultStr == '已经将米达尔从先攻列表中删除'
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr)
assert b.ProcessInput('.init clr', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr == '还没有做好先攻列表哦'
assert b.ProcessInput('.nn', personId = '12345678', personName = 'TEST_USER').resultStr == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'ABCD').resultStr[:11] == 'ABCD掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER').resultStr[:17] == 'TEST_USER今天走运的概率是'
assert b.ProcessInput('.sethp', '12345678', 'TEST_USER', 'groupId_test').resultStr == '已经忘记了TEST_USER的生命值...'

In [ ]:
assert b.ProcessInput('.nn 格鲁特', '12345678', 'TEST_USER').resultStr == '要称呼TEST_USER为格鲁特吗? 没问题!'
print(b.ProcessInput('.ri', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.ri', '87654321', '米达尔', 'groupId_test').resultStr)
print(b.ProcessInput('.ri 地精', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.ri 卓尔精灵A', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp 20/40', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp 地精- 1d8+4', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp 格+ 1d8+3', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp10', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp A-1d6+10', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethpA2d8', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethpA-20', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp米-3d4+3', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.init clr', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.sethp', '87654321', '米达尔', 'groupId_test').resultStr)
assert b.ProcessInput('.nn', '12345678', 'TEST_USER').resultStr == '要用本来的名字称呼你吗? 了解!'

In [ ]:
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test') == None
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test', True).resultStr == '来啦~ 让我看看哪个小可爱能得到今日份的礼物'
assert b.ProcessInput('.bot off', '12345678', 'TEST_USER', 'groupId_test', True).resultStr == '那我就不说话咯'
assert b.ProcessInput('.rd20沉默', '12345678', 'TEST_USER', 'groupId_test') == None
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test', True).resultStr == '来啦~ 让我看看哪个小可爱能得到今日份的礼物'
assert b.ProcessInput('.rd20沉默', '12345678', 'TEST_USER', 'groupId_test') != None

In [ ]:
assert b.ProcessInput('.dnd', '12345678', 'TEST_USER')
assert b.ProcessInput('.dnd 3', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.dnd3 大力出奇迹', '12345678', 'TEST_USER', 'groupId_test')

In [ ]:
assert b.ProcessInput('.help', '12345678', 'TEST_USER', 'groupId_test').resultStr == help_info.HELP_STR
assert b.ProcessInput('.help指令', '12345678', 'TEST_USER', 'groupId_test').resultStr == help_info.HELP_COMMAND_STR
assert b.ProcessInput('.help  指 令', '12345678', 'TEST_USER', 'groupId_test').resultStr == help_info.HELP_COMMAND_STR
assert b.ProcessInput('.help链接', '12345678', 'TEST_USER', 'groupId_test').resultStr == help_info.HELP_LINK_STR
assert b.ProcessInput('.help协议', '12345678', 'TEST_USER', 'groupId_test').resultStr == help_info.HELP_AGREEMENT_STR

In [ ]:
assert b.ProcessInput('.查询火球术', '12345678', 'TEST_USER', 'groupId_test').resultStr[:11] == '火球术Fireball'
assert b.ProcessInput('.查询  火球术', '12345678', 'TEST_USER', 'groupId_test').resultStr[:11] == '火球术Fireball'
assert b.ProcessInput('.查询  火 球 术', '12345678', 'TEST_USER', 'groupId_test').resultStr[:11] == '火球术Fireball'
assert b.ProcessInput('.查询火球', '12345678', 'TEST_USER', 'groupId_test').resultStr.find('找到多个匹配的条目:') != -1
assert b.ProcessInput('.查询   ', '12345678', 'TEST_USER', 'groupId_test')

In [ ]:
assert b.ProcessInput('.rhd20优势', '12345678', 'TEST_USER', 'groupId_test').personIdList == ['12345678']
assert b.ProcessInput('.rhd20优势', '12345678', 'TEST_USER', 'groupId_test').groupIdList is None

In [ ]:
print(b.ProcessInput('.r1d8抗性', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.r1d8+2d10抗性', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.r20易伤', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.rd+2d8+1d4易伤', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.r3#1d8+1d12抗性', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.r易伤d+2d8+1d4', '12345678', 'TEST_USER', 'groupId_test').resultStr)
print(b.ProcessInput('.r3dk1易伤', '12345678', 'TEST_USER', 'groupId_test').resultStr)

In [ ]:
# queryDict = {}

# fileName = '5E_护甲与武器_by邪恶'
# title = ''
# content = ''
# with open(f'../资料/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline()
#     while data:
#         data = data.strip()
#         if not data:
#             if title:
#                 if len(title) > 15:
#                     print("Title too long?"+title+"##"+content)
#                 elif len(content) < 20:
#                     print("Content too small?"+title+"##"+content)
#                 queryDict[title] = content[:-1]
#             title = ''
#             content = ''
#         else:
#             if not title:
#                 title = data
#             else:
#                 content += data.strip() + '\n'
#         data = f.readline()
# print(queryDict.keys())
# print(queryDict[list(queryDict.keys())[0]])
# utils.UpdateJson(queryDict, f'./{fileName}.json')